In [2]:
import laser_lib
import numpy as np
import math
import time
import cv2
from skimage.color import rgb2lab, lab2rgb

In [3]:
queue = laser_lib.DacQueue()

Found  1 Helios DACs


In [11]:
def signer(f, T, phase=0.0):
    '''
    Produces spaced out points from 0 to 1 whose distance between 
    consecutive points is sinusoidal 
    Input:  f - frequency 
            T - number of samples
            phase - phase shift (0, 1)
    Returns: x - positions on the line
             dx - the derivative of the function of x
    '''
    # Convert to radians
    f = f*2*np.pi
    # Generate the baseline 
    t = np.linspace(0.0+phase, 1.0+phase, T)
    # Find points 
    x = (f*t - np.cos(f*t) + 1)/f
    # Calculate derivative
    dx = (np.sin(f*t) + 1)/2
    
    return x, dx

def triangle(theta, alpha, sincos='sin'):
    '''
    Returns a wave between a sine wave and triangle wave depending on alpha
    alpha = 0 - sine wave
    alpha = 1 - triangle wave
    '''
    if(sincos == 'sin'):
        ps = 0
    elif(sincos == 'cos'):
        ps = np.pi/2
    return alpha*2*np.arcsin(np.sin(theta + ps))/np.pi + (1-alpha)*np.sin(theta + ps)
    
def ellipser(scale_x=1, scale_y=1, f_x=1, f_y=1, f_s=1000, phase_s=0, sintriang=0, colors=[(1,1,1)], T=500):
    '''
    Generates a pattern that is a function of ellipses 
    with parameters that can be interpolated between
    '''
    theta_norm, theta_dx = signer(f=f_s, phase=phase_s, T=T)
    theta = theta_norm*2*np.pi
    arr_pos = np.zeros((T, 2))
    arr_pos[:,0] = scale_x*triangle(f_x*theta, sintriang, 'sin')
    arr_pos[:,1] = scale_y*triangle(f_y*theta, sintriang, 'cos')
    
    # Color the pattern
    arr_col = np.ones((T, 3), np.float32)
    if(len(colors) == 1):
        # All one color
        arr_col[:, :] = colors[0][np.newaxis, :]
    if(len(colors) == 2):
        # Color based on speed of laser
        arr_col[theta_dx > 0.1, :] = colors[0]
        arr_col[theta_dx <= 0.1, :] = colors[1]
    
    return arr_pos, arr_col


In [4]:
# CIELAB COLOR SPACE
ab_min = -127
ab_max = 128
l_min, l_max  = (0, 130)
num_points = ab_max - ab_min + 1

l = np.linspace(l_min, l_max, num_points)
a = np.linspace(ab_min, ab_max, num_points)
b = np.linspace(ab_min, ab_max, num_points)
l, a, b = np.meshgrid(l, a, b, indexing='ij')
lab = np.stack([l, a, b], axis=-1)

# Convert LAB coords to RGB
rgb = lab2rgb(lab)

for lum in np.concatenate([np.arange(num_points), np.arange(num_points)[::-1]]):
    frame = rgb[lum, :, :, :]
    print('Lum', lum, lab[lum, 0])
    cv2.imshow('frame', cv2.resize(frame, None, fx=2, fy=2))
    cv2.waitKey(10)


/home/bij/anaconda3/envs/Laser/lib/python3.8/site-packages/skimage/color/colorconv.py:1128: UserWarning: Color data out of range: Z < 0 in 1496064 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


Lum 0 [[   0. -127. -127.]
 [   0. -127. -126.]
 [   0. -127. -125.]
 [   0. -127. -124.]
 [   0. -127. -123.]
 [   0. -127. -122.]
 [   0. -127. -121.]
 [   0. -127. -120.]
 [   0. -127. -119.]
 [   0. -127. -118.]
 [   0. -127. -117.]
 [   0. -127. -116.]
 [   0. -127. -115.]
 [   0. -127. -114.]
 [   0. -127. -113.]
 [   0. -127. -112.]
 [   0. -127. -111.]
 [   0. -127. -110.]
 [   0. -127. -109.]
 [   0. -127. -108.]
 [   0. -127. -107.]
 [   0. -127. -106.]
 [   0. -127. -105.]
 [   0. -127. -104.]
 [   0. -127. -103.]
 [   0. -127. -102.]
 [   0. -127. -101.]
 [   0. -127. -100.]
 [   0. -127.  -99.]
 [   0. -127.  -98.]
 [   0. -127.  -97.]
 [   0. -127.  -96.]
 [   0. -127.  -95.]
 [   0. -127.  -94.]
 [   0. -127.  -93.]
 [   0. -127.  -92.]
 [   0. -127.  -91.]
 [   0. -127.  -90.]
 [   0. -127.  -89.]
 [   0. -127.  -88.]
 [   0. -127.  -87.]
 [   0. -127.  -86.]
 [   0. -127.  -85.]
 [   0. -127.  -84.]
 [   0. -127.  -83.]
 [   0. -127.  -82.]
 [   0. -127.  -81.]
 [   0.

Lum 2 [[   1.01960784 -127.         -127.        ]
 [   1.01960784 -127.         -126.        ]
 [   1.01960784 -127.         -125.        ]
 [   1.01960784 -127.         -124.        ]
 [   1.01960784 -127.         -123.        ]
 [   1.01960784 -127.         -122.        ]
 [   1.01960784 -127.         -121.        ]
 [   1.01960784 -127.         -120.        ]
 [   1.01960784 -127.         -119.        ]
 [   1.01960784 -127.         -118.        ]
 [   1.01960784 -127.         -117.        ]
 [   1.01960784 -127.         -116.        ]
 [   1.01960784 -127.         -115.        ]
 [   1.01960784 -127.         -114.        ]
 [   1.01960784 -127.         -113.        ]
 [   1.01960784 -127.         -112.        ]
 [   1.01960784 -127.         -111.        ]
 [   1.01960784 -127.         -110.        ]
 [   1.01960784 -127.         -109.        ]
 [   1.01960784 -127.         -108.        ]
 [   1.01960784 -127.         -107.        ]
 [   1.01960784 -127.         -106.        ]
 [  

Lum 11 [[   5.60784314 -127.         -127.        ]
 [   5.60784314 -127.         -126.        ]
 [   5.60784314 -127.         -125.        ]
 [   5.60784314 -127.         -124.        ]
 [   5.60784314 -127.         -123.        ]
 [   5.60784314 -127.         -122.        ]
 [   5.60784314 -127.         -121.        ]
 [   5.60784314 -127.         -120.        ]
 [   5.60784314 -127.         -119.        ]
 [   5.60784314 -127.         -118.        ]
 [   5.60784314 -127.         -117.        ]
 [   5.60784314 -127.         -116.        ]
 [   5.60784314 -127.         -115.        ]
 [   5.60784314 -127.         -114.        ]
 [   5.60784314 -127.         -113.        ]
 [   5.60784314 -127.         -112.        ]
 [   5.60784314 -127.         -111.        ]
 [   5.60784314 -127.         -110.        ]
 [   5.60784314 -127.         -109.        ]
 [   5.60784314 -127.         -108.        ]
 [   5.60784314 -127.         -107.        ]
 [   5.60784314 -127.         -106.        ]
 [ 

Lum 20 [[  10.19607843 -127.         -127.        ]
 [  10.19607843 -127.         -126.        ]
 [  10.19607843 -127.         -125.        ]
 [  10.19607843 -127.         -124.        ]
 [  10.19607843 -127.         -123.        ]
 [  10.19607843 -127.         -122.        ]
 [  10.19607843 -127.         -121.        ]
 [  10.19607843 -127.         -120.        ]
 [  10.19607843 -127.         -119.        ]
 [  10.19607843 -127.         -118.        ]
 [  10.19607843 -127.         -117.        ]
 [  10.19607843 -127.         -116.        ]
 [  10.19607843 -127.         -115.        ]
 [  10.19607843 -127.         -114.        ]
 [  10.19607843 -127.         -113.        ]
 [  10.19607843 -127.         -112.        ]
 [  10.19607843 -127.         -111.        ]
 [  10.19607843 -127.         -110.        ]
 [  10.19607843 -127.         -109.        ]
 [  10.19607843 -127.         -108.        ]
 [  10.19607843 -127.         -107.        ]
 [  10.19607843 -127.         -106.        ]
 [ 

...

Lum 77 [[  39.25490196 -127.         -127.        ]
 [  39.25490196 -127.         -126.        ]
 [  39.25490196 -127.         -125.        ]
 [  39.25490196 -127.         -124.        ]
 [  39.25490196 -127.         -123.        ]
 [  39.25490196 -127.         -122.        ]
 [  39.25490196 -127.         -121.        ]
 [  39.25490196 -127.         -120.        ]
 [  39.25490196 -127.         -119.        ]
 [  39.25490196 -127.         -118.        ]
 [  39.25490196 -127.         -117.        ]
 [  39.25490196 -127.         -116.        ]
 [  39.25490196 -127.         -115.        ]
 [  39.25490196 -127.         -114.        ]
 [  39.25490196 -127.         -113.        ]
 [  39.25490196 -127.         -112.        ]
 [  39.25490196 -127.         -111.        ]
 [  39.25490196 -127.         -110.        ]
 [  39.25490196 -127.         -109.        ]
 [  39.25490196 -127.         -108.        ]
 [  39.25490196 -127.         -107.        ]
 [  39.25490196 -127.         -106.        ]
 [ 

Lum 86 [[  43.84313725 -127.         -127.        ]
 [  43.84313725 -127.         -126.        ]
 [  43.84313725 -127.         -125.        ]
 [  43.84313725 -127.         -124.        ]
 [  43.84313725 -127.         -123.        ]
 [  43.84313725 -127.         -122.        ]
 [  43.84313725 -127.         -121.        ]
 [  43.84313725 -127.         -120.        ]
 [  43.84313725 -127.         -119.        ]
 [  43.84313725 -127.         -118.        ]
 [  43.84313725 -127.         -117.        ]
 [  43.84313725 -127.         -116.        ]
 [  43.84313725 -127.         -115.        ]
 [  43.84313725 -127.         -114.        ]
 [  43.84313725 -127.         -113.        ]
 [  43.84313725 -127.         -112.        ]
 [  43.84313725 -127.         -111.        ]
 [  43.84313725 -127.         -110.        ]
 [  43.84313725 -127.         -109.        ]
 [  43.84313725 -127.         -108.        ]
 [  43.84313725 -127.         -107.        ]
 [  43.84313725 -127.         -106.        ]
 [ 

Lum 96 [[  48.94117647 -127.         -127.        ]
 [  48.94117647 -127.         -126.        ]
 [  48.94117647 -127.         -125.        ]
 [  48.94117647 -127.         -124.        ]
 [  48.94117647 -127.         -123.        ]
 [  48.94117647 -127.         -122.        ]
 [  48.94117647 -127.         -121.        ]
 [  48.94117647 -127.         -120.        ]
 [  48.94117647 -127.         -119.        ]
 [  48.94117647 -127.         -118.        ]
 [  48.94117647 -127.         -117.        ]
 [  48.94117647 -127.         -116.        ]
 [  48.94117647 -127.         -115.        ]
 [  48.94117647 -127.         -114.        ]
 [  48.94117647 -127.         -113.        ]
 [  48.94117647 -127.         -112.        ]
 [  48.94117647 -127.         -111.        ]
 [  48.94117647 -127.         -110.        ]
 [  48.94117647 -127.         -109.        ]
 [  48.94117647 -127.         -108.        ]
 [  48.94117647 -127.         -107.        ]
 [  48.94117647 -127.         -106.        ]
 [ 

Lum 105 [[  53.52941176 -127.         -127.        ]
 [  53.52941176 -127.         -126.        ]
 [  53.52941176 -127.         -125.        ]
 [  53.52941176 -127.         -124.        ]
 [  53.52941176 -127.         -123.        ]
 [  53.52941176 -127.         -122.        ]
 [  53.52941176 -127.         -121.        ]
 [  53.52941176 -127.         -120.        ]
 [  53.52941176 -127.         -119.        ]
 [  53.52941176 -127.         -118.        ]
 [  53.52941176 -127.         -117.        ]
 [  53.52941176 -127.         -116.        ]
 [  53.52941176 -127.         -115.        ]
 [  53.52941176 -127.         -114.        ]
 [  53.52941176 -127.         -113.        ]
 [  53.52941176 -127.         -112.        ]
 [  53.52941176 -127.         -111.        ]
 [  53.52941176 -127.         -110.        ]
 [  53.52941176 -127.         -109.        ]
 [  53.52941176 -127.         -108.        ]
 [  53.52941176 -127.         -107.        ]
 [  53.52941176 -127.         -106.        ]
 [

KeyboardInterrupt: 

In [ ]:
# Sampling the CIELAB Space
AB_choices = np.arange(start, stop+1).astype(np.float32)
while(True):
    # Choose a Lum
    lum_c = int(100*np.random.rand(1))
    # Choose an A
    a_c = np.random.choice(AB_choices)
    # Choose a B 
    b_c = np.random.choice(AB_choices)

    try:
        point_rgb = lab2rgb([[lum_c, a_c, b_c]])
        point_rgb_opp = lab2rgb((lum_c, -1*a_c, -1*b_c))
    except:
        # Skip this invalid combo
        continue
    print((lum_c, a_c, b_c), point_rgb)
    

    frame = np.zeros((400, 200, 3))
    frame[0:200, :, :] = point_rgb
    frame[200:400, :, :] = point_rgb_opp
    cv2.imshow('point', frame)
    cv2.waitKey(1000)


In [5]:
# Make spiral of CIELab
T = 1000
turns = 7
theta = np.linspace(0, 2*np.pi*turns, T)
scale = np.linspace(0, 1, T)

arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = scale*np.cos(theta)
arr_pos[:, 1] = scale*np.sin(theta)

arr_col_lab = np.zeros((T, 3))
arr_col_lab[:, 0] = (l_max-l_min)*np.sqrt(arr_pos[:, 0]**2 + arr_pos[:, 1]**2) + l_min
w = (l_max-l_min)*scale + l_min
arr_col_lab[:, 1] = (ab_max-ab_min)*(arr_pos[:, 0]+1)/2 + ab_min
arr_col_lab[:, 2] = (ab_max-ab_min)*(arr_pos[:, 1]+1)/2 + ab_min

arr_col_rgb = lab2rgb(arr_col_lab)

queue.dac_rate = 40000
for step in range(1000):
    queue.submit(arr_pos, arr_col_rgb)
    #queue.submit(arr_pos[::-1], arr_col_rgb[::-1])

In [11]:
# Circle
queue.dac_rate = 30000
T = 500
arr_pos = np.zeros((T, 2))
arr_col = np.zeros((T, 3))
arr_col[:, :] = 1
theta = np.linspace(0, 2*np.pi, T)
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = arr_col*np.expand_dims((theta > np.pi/2), 1)

while(True):
    queue.submit(arr_pos, arr_col/6)

KeyboardInterrupt: 

In [6]:
# Interpolated ellipses
T = 1000
arr_col = np.ones((T, 3))
theta = np.linspace(0, 2*np.pi, T)

queue.dac_rate = 45000

min_vals = np.array([0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
max_vals = np.array([1.0, 0.4, 2.0, 3.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
pat_point_old = (max_vals-min_vals)*np.random.rand(len(max_vals)) + min_vals


T_trans = 150

while(True):
    pat_point = (max_vals-min_vals)*np.random.rand(len(max_vals)) + min_vals
    ell_interp = np.linspace(pat_point_old, pat_point, T_trans)
    for (scale_x, scale_y, freq_x, freq_y, f_s, phase, sintriang, r1, g1, b1, r2, g2, b2) in ell_interp:
        # Transform the distribution of f_s to be less uniform
        f_s = 40*np.power(f_s, 6)
        #TODO - transform sintriang to have a bathtub curve
        
        # Get the array
        arr_pos, arr_col = ellipser(scale_x=scale_x, 
                                    scale_y=scale_y, 
                                    f_x=freq_x, 
                                    f_y=freq_y, 
                                    f_s=f_s, 
                                    phase_s=phase,
                                    sintriang=sintriang, 
                                    colors=[(r1,g1,b1), (r2,g2,b2)],
                                    T=T)
        
        queue.submit(arr_pos, np.roll(arr_col, shift=9, axis=0))
        
    pat_point_old = pat_point

KeyboardInterrupt: 

In [18]:
# Cycling back and forth between ellipses
T = 1000
queue.dac_rate = 45000

min_vals = np.array([0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
max_vals = np.array([1.0, 0.4, 2.0, 3.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])


T_trans = 50

repeat_cycle_count = 4

while(True):
    # Pick two points to cycle around
    pat_point_a = (max_vals-min_vals)*np.random.rand(len(max_vals)) + min_vals
    pat_point_b = (max_vals-min_vals)*np.random.rand(len(max_vals)) + min_vals
    # Go there and come back
    ell_interp = np.linspace(pat_point_a, pat_point_b, T_trans//2)
    ell_interp = np.concatenate([ell_interp, ell_interp[::-1, :]])
    for r in range(repeat_cycle_count):
        for (scale_x, scale_y, freq_x, freq_y, f_s, phase, sintriang, r1, g1, b1, r2, g2, b2) in ell_interp:
            # Transform the distribution of f_s to be less uniform
            f_s = 40*np.power(f_s, 6)
            #TODO - transform sintriang to have a bathtub curve

            # Get the array
            arr_pos, arr_col = ellipser(scale_x=scale_x, 
                                        scale_y=scale_y, 
                                        f_x=freq_x, 
                                        f_y=freq_y, 
                                        f_s=f_s, 
                                        phase_s=phase,
                                        sintriang=sintriang, 
                                        colors=[(r1,g1,b1), (r2,g2,b2)],
                                        T=T)

            queue.submit(arr_pos, np.roll(arr_col, shift=9, axis=0))
        


(25, 13) (25, 13)
(25, 13) (25, 13)
(25, 13) (25, 13)
(25, 13) (25, 13)


KeyboardInterrupt: 

In [18]:
T = 750
arr_col = np.ones((T, 3))
theta = np.linspace(0, 2*np.pi, T)



(r1,g1,b1) = (0, 0.1, 0.1)
(r2,g2,b2) = (1.0, 0.4, 0.4)
for shift in range(-10, 10):
    print(shift)
    for i in range(100):
        arr_pos, arr_col = ellipser(scale_x=1.0, 
                                    scale_y=1.0, 
                                    f_x=1.0, 
                                    f_y=1.0, 
                                    f_s=10, 
                                    phase_s=0.0,
                                    sintriang=0.0, 
                                    colors=[(r1,g1,b1), (r2,g2,b2)],
                                    T=T)

        queue.submit(arr_pos, np.roll(arr_col, shift, axis=0))
        


-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0


KeyboardInterrupt: 

In [ ]:
# Align mirrors and laser 
T = 1000
turns = 7
theta = np.linspace(0, 2*np.pi*turns, T)

arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.linspace(-1.0, 1.0, T)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.zeros((T, 3))
#arr_col[:, 0] = np.abs(arr_pos[:, 1]) > 0.99
arr_col[:, 0] = np.abs(arr_pos[:, 1]) > 0.999
arr_col[:, 1] = np.abs(arr_pos[:, 1]) > 0.999
arr_col[:, 2] = np.abs(arr_pos[:, 1]) > 0.999

# Corresponds to Red, Green, Blue @ dac_rate 55k
shifts = [11, 10, 9]

# Correct color array by shifting
#for col_idx  in range(3):
#    arr_col[:, col_idx] = np.roll(arr_col[:, col_idx], shifts[col_idx], axis=0)
    
#for col_idx, shift in enumerate(shifts):
#    print(col_idx, shift)
#    arr_col[:, col_idx] = np.roll(arr_col[:, col_idx], shift, axis=0)

for shift in range(8, 20):
    print(shift)
    for i in range(100):
        queue.submit(arr_pos, arr_col)

In [ ]:
# Circle
queue.dac_rate = 30000
T = 500
arr_pos = np.zeros((T, 2))
arr_col = np.zeros((T, 3))
arr_col[:, :] = 1
theta = np.linspace(0, 2*np.pi, T)
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)


while(True):
    queue.submit(arr_pos, arr_col/6)